In [1]:
import pandas as pd

In [2]:
df = pd.read_spss('Climate Anxiety-3.SAV')
df

,Respondent_Serial,language,country,D1,D2,D3,Q1,Q2_1,Q2_2,Q2_3,...,regionUS,regionUK,regionIndia,regionNigeria,regionPhilippines,regionFinland,regionPortugal,regionBrazil,regionFrance,weight
0,25.0,French,France,22.0,man,France: South,extremely,yes,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South,0.955127
1,26.0,English,Philippines,25.0,woman,Philippines: Metro Luzon Urban Beltway/ North ...,extremely,yes,yes,yes,...,NaN,NaN,NaN,NaN,Metro Luzon Urban Beltway/ North Luzon Agribus...,NaN,NaN,NaN,NaN,0.649680
2,27.0,English,Philippines,24.0,woman,Philippines: Metro Luzon Urban Beltway/ North ...,extremely,yes,yes,yes,...,NaN,NaN,NaN,NaN,Metro Luzon Urban Beltway/ North Luzon Agribus...,NaN,NaN,NaN,NaN,0.649680
3,28.0,English,Australia,20.0,woman,Australia: South Australia,a little,no,yes,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.915752
4,29.0,English,India,18.0,woman,India: West India,extremely,no,no,no,...,NaN,NaN,West India,NaN,NaN,NaN,NaN,NaN,NaN,1.183360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15413.0,English,Australia,21.0,woman,Australia: Tasmania,extremely,yes,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.739153
9996,15415.0,English,Australia,25.0,woman,Australia: New South Wales,very,yes,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.771648
9997,15416.0,English,Australia,19.0,woman,Australia: New South Wales,extremely,yes,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.869418
9998,15417.0,English,Australia,21.0,woman,Australia: Western Australia,very,yes,yes,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.821953
